In [34]:
import plotly.graph_objects as go
import shinybroker as sb
import numpy as np

asset = sb.Contract({
    'symbol': "NVDA",
    'secType': "STK",
    'exchange': "SMART",
    'currency': "USD"
})

benchmark = sb.Contract({
    'symbol': "VOO",
    'secType': "STK",
    'exchange': "SMART",
    'currency': "USD"
})

def fetch_security_historical_data(security_info):
    historical_data = sb.fetch_historical_data(
        security_info,
        barSizeSetting="1 day",
        durationStr="1 Y"
    )
    return historical_data["hst_dta"]["close"]

def calculate_geometric_returns(close_prices):
    close_prices = np.array(close_prices)
    geometric_returns = np.log(close_prices[1:] / close_prices[:-1])
    return geometric_returns

def calculate_arithmetic_returns(close_prices):
    close_prices = np.array(close_prices)
    arithmetic_returns = np.diff(close_prices) / close_prices[:-1]
    return arithmetic_returns

def calculate_beta(asset_return, benchmark_return):
    cov = np.cov(asset_return, benchmark_return)
    cov_xy = cov[0, 1]
    var = np.var(benchmark_return, ddof=1)
    beta = cov_xy / var
    return beta

def calculate_alpha(asset_return, benchmark_return):
    mean_asset_return = np.mean(asset_return)
    mean_benchmark_return = np.mean(benchmark_return)
    beta = calculate_beta(asset_return, benchmark_return)
    alpha = mean_asset_return - beta * mean_benchmark_return
    return alpha

def calculate_best_fit_range(data1, data2, padding_factor=0.1):
    combined_data = np.concatenate((data1, data2))
    data_min = np.min(combined_data)
    data_max = np.max(combined_data)
    range_padding = (data_max - data_min) * padding_factor
    best_min = data_min - range_padding
    best_max = data_max + range_padding
    range_span = best_max - best_min
    tick_increment = round(range_span / 10, 4)  # Divide range into ~10 ticks
    return best_min, best_max, tick_increment
    

# Fetch data for both securities
asset_close_data = fetch_security_historical_data(asset)
benchmark_close_data = fetch_security_historical_data(benchmark)

# Calculate the geometric and arithemetic returns 
asset_geometric_returns = calculate_geometric_returns(asset_close_data)
benchmark_geometric_returns = calculate_geometric_returns(benchmark_close_data)
asset_arithmetic_returns = calculate_arithmetic_returns(asset_close_data)
benchmark_arithmetic_returns = calculate_arithmetic_returns(benchmark_close_data)

# Calculate alpha/beta for geometric and arithmetic returns
geo_beta = calculate_beta(asset_geometric_returns, benchmark_geometric_returns)
geo_alpha = calculate_alpha(asset_geometric_returns, benchmark_geometric_returns)
arith_beta = calculate_beta(asset_arithmetic_returns, benchmark_arithmetic_returns)
arith_alpha = calculate_alpha(asset_arithmetic_returns, benchmark_arithmetic_returns)

print(f"Arithmetic - alpha: {arith_alpha} | beta: {arith_beta}")
print(f"Geometric - alpha: {geo_alpha} | beta: {geo_beta}")



Arithmetic - alpha: 0.002018365817306291 | beta: 2.6513212487252464
Geometric - alpha: 0.0015594436906898827 | beta: 2.634653492907397


In [35]:
def create_alpha_beta_plot(asset_returns, benchmark_returns, alpha, beta, title, color, line_color, x_range=None, y_range=None, tick_increment=None):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=benchmark_returns,
        y=asset_returns,
        mode='markers',
        name='Returns',
        marker=dict(size=7, color=color, opacity=0.7)
    ))

    trendline = beta * benchmark_returns + alpha
    fig.add_trace(go.Scatter(
        x=benchmark_returns,
        y=trendline,
        mode='lines',
        name=f'Trendline (Beta={beta:.2f}, Alpha={alpha:.2f})',
        line=dict(color=line_color, dash='solid')
    ))

    layout = {
        "title": title,
        "xaxis": {
            "title": "Benchmark Returns",
            "range": x_range,
            "tickmode": "linear" if tick_increment else "auto",
            "dtick": tick_increment if tick_increment else None,
        },
        "yaxis": {
            "title": "Asset Returns",
            "range": y_range,
            "tickmode": "linear" if tick_increment else "auto",
            "dtick": tick_increment if tick_increment else None,
        },
        "legend_title": "Legend",
        "template": "plotly_white"
    }
    fig.update_layout(**layout)

    return fig

# Plot for geometric returns
geo_fig = create_alpha_beta_plot(
    asset_returns=asset_geometric_returns,
    benchmark_returns=benchmark_geometric_returns,
    alpha=geo_alpha,
    beta=geo_beta,
    title="Geometric Returns Alpha and Beta",
    color="blue",
    line_color="red",
    x_range = (-0.05, 0.05),
    y_range= (-0.05, 0.05),
    tick_increment=0.02
)

# Plot for arithmetic returns
arith_fig = create_alpha_beta_plot(
    asset_returns=asset_arithmetic_returns,
    benchmark_returns=benchmark_arithmetic_returns,
    alpha=arith_alpha,
    beta=arith_beta,
    title="Arithmetic Returns Alpha and Beta",
    color="green",
    line_color="orange",
    x_range = (-0.05, 0.05),
    y_range= (-0.05, 0.05),
    tick_increment=0.02
)

geo_fig.show()
arith_fig.show()